# Generar tabla en excel con temporizador

In [ ]:
# Librerías a instalar
#!pip install schedule xlsxwriter pandas openpyxl

In [1]:
# Importar librerías
import re
import glob
import os
import glob


import os
import html
import pandas as pd
import openpyxl
import schedule
import time
import pickle
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime

# Rutas de archivos y directorios
current_dir = os.getcwd()
csv_oc_pendientes = os.path.join(current_dir, 'OCS\\OCs_Pendientes.csv')
pickle_file = os.path.join(current_dir, 'OCS\\facturas_procesadas.pickle')
ruta_excel_salida = os.path.join(current_dir, 'OCS\\salida.xlsx')
ruta_terceros_csv = os.path.join(current_dir, 'OCS\\terceros.csv')

#Lectura de claves:
def obtener_clave():
    """
    Esta función se encarga de leer y retornar la clave de acceso a Gmail almacenada en un archivo.

    La clave es almacenada en un archivo de texto plano ('gmailKey') en el directorio de ejecución
    del script. Este enfoque permite separar las credenciales del código, mejorando la seguridad y
    facilitando la configuración en diferentes entornos sin necesidad de modificar el código fuente.

    Retorna:
        gmailKey (str): Una cadena de texto que contiene la clave de acceso a Gmail.
    """
    with open('gmailKey', 'r') as file:  # Abre el archivo 'gmailKey' en modo lectura.
        gmailKey = file.read().strip()  # Lee la clave del archivo, eliminando espacios en blanco y saltos de línea.
    return gmailKey  # Retorna la clave leída.

# Funciones principales
def enviar_correo(asunto, cuerpo, destinatario, adjuntos=[]):
    """
    Envía un correo electrónico a un destinatario específico, con la opción de incluir archivos adjuntos.

    Utiliza el protocolo SMTP para la comunicación con el servidor de correo de Gmail. La función soporta
    el envío de múltiples archivos adjuntos, permitiendo una amplia gama de aplicaciones como enviar
    reportes, facturas, notificaciones, etc.

    Parámetros:
        asunto (str): El asunto del correo electrónico.
        cuerpo (str): El cuerpo del correo electrónico, en texto plano.
        destinatario (str): La dirección de correo electrónico del destinatario.
        adjuntos (list, opcional): Una lista de rutas de archivos que se adjuntarán al correo. Por defecto, es una lista vacía.

    Notas:
        - Es necesario tener un archivo 'gmailKey' en el directorio de ejecución con la contraseña de la
            cuenta de correo del emisor.
        - La dirección del emisor está codificada en la función; es recomendable externalizarla o
            parametrizarla para aumentar la flexibilidad y seguridad.
    """
    emisor = 'aaguanangap@gmail.com'  # Dirección de correo electrónico del emisor.
    contraseña = obtener_clave()  # Obtiene la contraseña del archivo 'gmailKey'.

    mensaje = MIMEMultipart()  # Crea un objeto MIMEMultipart para el mensaje.
    mensaje['From'] = emisor  # Establece el emisor.
    mensaje['To'] = destinatario  # Establece el destinatario.
    mensaje['Subject'] = asunto  # Establece el asunto del correo.

    mensaje.attach(MIMEText(cuerpo, 'plain'))  # Adjunta el cuerpo del mensaje como texto plano.

    for archivo in adjuntos:  # Procesa cada archivo adjunto.
        parte = MIMEBase('application', 'octet-stream')  # Crea un objeto MIMEBase para el archivo.
        with open(archivo, 'rb') as file:  # Abre el archivo en modo binario.
            parte.set_payload(file.read())  # Lee y adjunta el contenido del archivo.
        encoders.encode_base64(parte)  # Codifica el contenido en base64.
        parte.add_header('Content-Disposition', f"attachment; filename= {os.path.basename(archivo)}")  # Añade el nombre del archivo.
        mensaje.attach(parte)  # Adjunta el archivo al mensaje.

    server = smtplib.SMTP('smtp.gmail.com', 587)  # Conecta al servidor de Gmail usando SMTP en el puerto 587.
    server.starttls()  # Inicia TLS para la seguridad de la conexión.
    server.login(emisor, contraseña)  # Inicia sesión con las credenciales del emisor.
    text = mensaje.as_string()  # Convierte el mensaje a una cadena de texto.
    server.sendmail(emisor, destinatario, text)  # Envía el correo.
    server.quit()  # Cierra la conexión con el servidor.

def cargar_o_inicializar_registros():
    """
    Carga los registros almacenados en un archivo pickle o inicializa nuevos registros si el archivo no existe.

    Esta función intenta cargar un diccionario desde un archivo pickle especificado en la variable global
    'pickle_file'. Si el archivo no existe o está vacío (lo que podría indicar un archivo dañado o no inicializado),
    se retorna un nuevo diccionario con estructuras para almacenar facturas procesadas y carpetas vacías.

    Retorna:
        Un diccionario con dos claves: 'facturas_procesadas' y 'carpetas_vacias'. Cada una de estas claves
        apunta a un diccionario vacío si es la primera ejecución o no se encuentra el archivo pickle. Si el archivo
        existe y contiene datos, retorna los datos almacenados previamente.

    Excepciones:
        FileNotFoundError: Si el archivo especificado en 'pickle_file' no existe.
        EOFError: Si el archivo 'pickle_file' existe pero está vacío o dañado.

    Nota:
        Esta función depende de la variable global 'pickle_file' que debe contener la ruta al archivo pickle
        donde se guardan los registros. Es fundamental asegurar que esta variable esté definida antes de
        llamar a la función.
    """
    try:
        with open(pickle_file, 'rb') as f:  # Intenta abrir el archivo pickle en modo lectura binaria.
            return pickle.load(f)  # Retorna el diccionario cargado desde el archivo pickle.
    except (FileNotFoundError, EOFError):  # Captura errores si el archivo no existe o está vacío.
        return {"facturas_procesadas": {}, "carpetas_vacias": {}}  # Retorna un nuevo diccionario con estructuras iniciales vacías.

def registrar_carpetas_vacias():
    """
    Identifica y registra las subcarpetas dentro de un directorio específico que no contienen archivos XML,
    marcándolas como carpetas vacías en los registros. Actualiza estos registros en un archivo pickle y
    actualiza el archivo CSV de OCs pendientes basado en las carpetas vacías encontradas.

    Este proceso implica las siguientes etapas:
    - Cargar o inicializar los registros existentes.
    - Buscar en un directorio predefinido para identificar todas las subcarpetas.
    - Para cada subcarpeta, verificar si contiene archivos con la extensión '.xml'.
    - Si una subcarpeta no contiene archivos '.xml', se considera 'vacía' y se registra su nombre (generalmente,
        un número de Orden de Compra, OC) en los registros bajo la clave "carpetas_vacias".
    - Los registros actualizados se guardan de nuevo en el archivo pickle.
    - Finalmente, se actualiza un archivo CSV que lista las OCs pendientes basadas en las carpetas identificadas como vacías.

    Nota:
    - La función depende de variables globales para las rutas de archivos y directorios, como 'pickle_file',
        para la ruta al archivo pickle, y 'csv_oc_pendientes' para la ruta al archivo CSV de OCs pendientes.
    - La función 'actualizar_csv_oc_pendientes' se llama al final para actualizar el archivo CSV con las OCs
        pendientes basado en las carpetas vacías registradas.
    """
    registros = cargar_o_inicializar_registros()  # Carga o inicializa los registros.
    subcarpetas = [d for d in glob.glob(os.path.join(current_dir, 'OCS\\*\\'))]  # Lista todas las subcarpetas.
    
    for carpeta in subcarpetas:
        if not any(f.endswith('.xml') for f in os.listdir(carpeta)):  # Si no hay archivos .xml en la carpeta.
            oc = os.path.basename(carpeta.rstrip('\\'))  # Extrae el nombre de la OC basado en el nombre de la carpeta.
            registros["carpetas_vacias"][oc] = True  # Marca la OC como carpeta vacía en los registros.

    with open(pickle_file, 'wb') as f:  # Abre el archivo pickle en modo escritura binaria.
        pickle.dump(registros, f)  # Guarda los registros actualizados en el archivo pickle.

    # Actualiza el archivo CSV con las OCs pendientes basado en las carpetas vacías registradas.
    actualizar_csv_oc_pendientes(registros["carpetas_vacias"].keys())

def actualizar_csv_oc_pendientes(ocs):
    """
    Actualiza el archivo CSV que lista las Órdenes de Compra (OCs) pendientes, basándose en un conjunto de OCs
    proporcionado como argumento. Esta función es típicamente llamada después de identificar carpetas vacías o
    después de procesar las facturas, para reflejar el estado actual de las OCs en el sistema.

    Parámetros:
        ocs (iterable): Un iterable que contiene los números de las Órdenes de Compra que se consideran pendientes.
                        Cada elemento del iterable es una cadena de texto que representa el número de una OC.

    Funcionamiento:
        - Crea un DataFrame de pandas con una columna 'OC', donde cada fila representa una OC pendiente.
        - Guarda este DataFrame en un archivo CSV, sobrescribiendo cualquier contenido previo. Esto asegura que el archivo
            refleje el estado más reciente de las OCs pendientes.
        - La ruta al archivo CSV es determinada por la variable global 'csv_oc_pendientes', la cual debe estar definida
            previamente y contener la ruta completa al archivo destino.

    Notas:
        - La función no retorna ningún valor.
        - Es importante asegurarse de que el archivo CSV destino esté accesible y que la aplicación tenga permisos
            suficientes para escribir en la ubicación especificada.
        - Esta función depende de la biblioteca pandas para la creación del DataFrame y la escritura del archivo CSV.
    """
    df_oc_pendientes = pd.DataFrame({"OC": list(ocs)})  # Crea un DataFrame con las OCs pendientes.
    df_oc_pendientes.to_csv(csv_oc_pendientes, index=False)  # Guarda el DataFrame en un archivo CSV, sin índice.

def limpiar_registros_carpetas():
    """
    Revisa y limpia los registros de carpetas marcadas como vacías en caso de que ahora contengan archivos XML.
    Este proceso ayuda a mantener actualizada la información sobre las carpetas que realmente están vacías y aquellas
    que ya no lo están, debido a la adición de archivos XML posteriormente.

    Funcionamiento:
        - Carga los registros existentes, incluyendo las carpetas marcadas previamente como vacías.
        - Revisa cada una de las carpetas registradas como vacías para verificar si ahora contienen archivos XML.
        - Si se encuentran archivos XML en una carpeta previamente marcada como vacía, se elimina dicha carpeta de los
            registros de carpetas vacías.
        - Actualiza el archivo pickle con los registros limpiados.
        - Actualiza el archivo CSV de OCs pendientes para reflejar los cambios en los registros de carpetas vacías.

    Notas:
        - La función depende de las variables globales 'pickle_file' para la ruta al archivo pickle y
            'csv_oc_pendientes' para la ruta al archivo CSV de OCs pendientes. Estas rutas deben estar correctamente
            establecidas antes de llamar a la función.
        - Se asume que los nombres de las carpetas corresponden a los números de las Órdenes de Compra (OCs),
            y que los archivos XML dentro de estas carpetas son relevantes para el proceso de facturación o
            registro correspondiente.
    """
    registros = cargar_o_inicializar_registros()  # Carga los registros actuales.
    carpetas_a_eliminar = [carpeta for carpeta in registros["carpetas_vacias"] if any(f.endswith('.xml') for f in os.listdir(os.path.join(current_dir, 'OCS', carpeta)))] #Identifica carpetas a limpiar
                            
    for carpeta in carpetas_a_eliminar:  # Elimina las entradas de carpetas que ya no están vacías.
        del registros["carpetas_vacias"][carpeta]

    with open(pickle_file, 'wb') as f:  # Guarda los registros actualizados en el archivo pickle.
        pickle.dump(registros, f)

    actualizar_csv_oc_pendientes(registros["carpetas_vacias"].keys())  # Actualiza el archivo CSV de OCs pendientes.

def normalizar_ruc(ruc, longitud_estandar=13):
    # Asegura que el RUC tenga la longitud estándar, añadiendo ceros al inicio si es necesario
    return ruc.zfill(longitud_estandar)

def extraer_informacion_de_archivo(ruta_archivo):
    """
    Extrae información relevante de un archivo de factura XML, incluyendo datos del emisor, la factura y descripciones de productos.

    Esta función intenta leer el contenido de un archivo XML especificado por 'ruta_archivo', buscando extraer datos específicos
    mediante expresiones regulares. Estos datos incluyen el RUC, establecimiento, punto de emisión, secuencial de factura,
    total sin impuestos, fecha de emisión, nombre comercial, y detalles de productos o servicios facturados.

    Parámetros:
        ruta_archivo (str): La ruta al archivo XML del que se desea extraer la información.

    Retorna:
        Un diccionario con los datos extraídos, que incluye claves como 'ruc', 'estab', 'ptoEmi', 'secuencial',
        'total_sin_impuestos', 'fecha_emision', 'nombre_comercial', 'OC' (orden de compra), 'Tercero' (identificado mediante RUC),
        y 'descripciones' (detalles de productos o servicios con sus precios y cantidades).

    Notas:
        - La función maneja dos codificaciones posibles para la lectura de archivos: UTF-8 y ISO-8859-1,
            para asegurar compatibilidad con diferentes formatos de archivos XML.
        - Utiliza 'html.unescape' para convertir entidades HTML a sus caracteres correspondientes, mejorando la lectura de los datos.
        - Implementa una serie de expresiones regulares para extraer la información deseada del contenido del archivo.
        - Normaliza los RUCs y utiliza el mapeo de terceros (cargado desde 'terceros.csv') para añadir el valor de 'Tercero'
            al diccionario de datos extraídos.
        - Concatena descripciones de productos o servicios con sus precios y cantidades, proporcionando una visión detallada
            de los ítems facturados.
    """
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            contenido = archivo.read()
    except UnicodeDecodeError:
        with open(ruta_archivo, 'r', encoding='ISO-8859-1', errors='replace') as archivo:
            contenido = archivo.read()

    contenido = html.unescape(contenido)

    patrones = {
        'ruc': r'<ruc>(.*?)<\/ruc>',
        'estab': r'<estab>(.*?)<\/estab>',
        'ptoEmi': r'<ptoEmi>(.*?)<\/ptoEmi>',
        'secuencial': r'<secuencial>(.*?)<\/secuencial>',
        'total_sin_impuestos': r'<totalSinImpuestos>(.*?)<\/totalSinImpuestos>',
        'fecha_emision': r'<fechaEmision>(.*?)<\/fechaEmision>',
        'nombre_comercial': r'<razonSocial>(.*?)<\/razonSocial>',
        'compania': r'<razonSocialComprador>(.*?)<\/razonSocialComprador>'
    }

    datos_extraidos = {}
    for clave, patron in patrones.items():
        coincidencia = re.search(patron, contenido, re.DOTALL)
        if coincidencia:
            datos_extraidos[clave] = coincidencia.group(1)
        else:
            datos_extraidos[clave] = 'No Disponible'

    if datos_extraidos['fecha_emision'] != 'No Disponible':
        datos_extraidos['fecha_formateada'] = datos_extraidos['fecha_emision']
    else:
        datos_extraidos['fecha_formateada'] = 'No Disponible'

    datos_extraidos['OC'] = os.path.basename(os.path.dirname(ruta_archivo))

    ruc = datos_extraidos.get('ruc')
    ruc_normalizado = normalizar_ruc(ruc)
    mapeo_terceros = cargar_y_mapear_terceros(ruta_terceros_csv)
    #datos_extraidos['Tercero'] = cargar_y_mapear_terceros(ruta_terceros_csv).get(ruc_normalizado, 'No Disponible')
    datos_extraidos['Tercero'] = mapeo_terceros.get(ruc_normalizado, {}).get('TERCERO', 'No Disponible')
    datos_extraidos['Centro de Costo'] = mapeo_terceros.get(ruc_normalizado, {}).get('CC', 'No Disponible')
    datos_extraidos['Nombre Farmacia'] = mapeo_terceros.get(ruc_normalizado, {}).get('NOMBRE FARMACIA', 'No Disponible')
    datos_extraidos['Frecuancia facturación'] = mapeo_terceros.get(ruc_normalizado, {}).get('FACTURA SEMESTRAL/MENSUAL', 'No Disponible')

    descripcion_tags = re.findall(r'<descripcion>(.*?)<\/descripcion>', contenido)
    precio_unitario_tags = re.findall(r'<precioUnitario>(.*?)<\/precioUnitario>', contenido)
    cantidad_tags = re.findall(r'<cantidad>(.*?)<\/cantidad>', contenido)

    precio_unitario_redondeado = [f"{float(precio):.2f}" for precio in precio_unitario_tags]
    cantidad_redondeado = [f"{float(cantidad):.2f}" for cantidad in cantidad_tags]

    descriptions_with_prices = [
        f"{descripcion} ({precio_unitario} x {cantidad})"
        for descripcion, precio_unitario, cantidad in zip(descripcion_tags, precio_unitario_redondeado, cantidad_redondeado)
    ]

    datos_extraidos['descripciones'] = " - ".join(descriptions_with_prices)

    return datos_extraidos

def actualizar_tabla_excel_y_limpieza(ruta_excel_salida):
    """
    Busca en un directorio predefinido archivos XML, extrae información relevante de cada uno, y actualiza
    un archivo Excel con esta información. Además, realiza una limpieza de registros, actualizando tanto
    un archivo pickle con facturas procesadas como un archivo CSV con OCs pendientes.

    Parámetros:
        ruta_excel_salida (str): La ruta al archivo Excel donde se guardarán los datos extraídos de los archivos XML.

    Funcionamiento:
        - Busca de manera recursiva archivos XML en el directorio de facturas.
        - Para cada archivo encontrado, extrae información como el RUC, nombre comercial, datos de factura, etc.
        - Si la factura no ha sido procesada previamente (según el registro en el archivo pickle), la agrega a un
            DataFrame para su posterior inclusión en el archivo Excel.
        - Envía correos electrónicos notificando sobre las facturas procesadas que no estaban previamente registradas.
        - Actualiza el archivo Excel con las nuevas facturas procesadas, el archivo pickle con el registro de facturas,
            y el archivo CSV con las OCs pendientes.

    Notas:
        - La función depende de variables globales para las rutas de archivos y directorios, como 'pickle_file' y
            'csv_oc_pendientes', que deben estar definidas y contener las rutas correctas.
        - Utiliza pandas para la manipulación de datos y openpyxl para la actualización de archivos Excel.
        - Esta función se asegura de que solo las facturas no registradas previamente sean procesadas y notificadas,
            evitando duplicados en el archivo Excel y en las notificaciones enviadas.
    """
    archivos = glob.glob(os.path.join(current_dir, 'OCS', '**', '*.xml'), recursive=True)
    dataframe_total = pd.DataFrame()

    with open(pickle_file, 'rb') as f:
        facturas_procesadas = pickle.load(f)
    df_oc_pendientes = pd.read_csv(csv_oc_pendientes)

    for ruta_archivo in archivos:
        informacion = extraer_informacion_de_archivo(ruta_archivo)
        factura = f"{informacion['estab']}-{informacion['ptoEmi']}-{informacion['secuencial']}"
        oc = informacion['OC']

        if factura not in facturas_procesadas:
            facturas_procesadas[factura] = True
            descripcion = informacion['descripciones']
            dataframe_temporal = pd.DataFrame({
                'RUC': [informacion['ruc']],
                'Tercero': [informacion['Tercero']],
                'Nombre Comercial': [informacion['nombre_comercial']],
                'Compañía': [informacion['compania']],
                'Centro de Costo': [informacion['Centro de Costo']],
                'Nombre Farmacia': [informacion['Nombre Farmacia']],
                'Factura': [factura],
                'Total Sin Impuestos': [informacion['total_sin_impuestos']],
                'Fecha': [informacion['fecha_formateada']],
                'OC': [informacion['OC']],
                'Frecuancia facturación': [informacion['Frecuancia facturación']],
                'Descripcion': [descripcion],
            })

            # Añadir la fecha de envío del correo al DataFrame
            fecha_envio_correo = pd.to_datetime('today').strftime('%Y-%m-%d')
            dataframe_temporal['Fecha de Envío Correo'] = fecha_envio_correo

            dataframe_total = pd.concat([dataframe_total, dataframe_temporal], ignore_index=True)

            if oc in facturas_procesadas and not facturas_procesadas[oc]:
                facturas_procesadas[oc] = True
                df_oc_pendientes = df_oc_pendientes[df_oc_pendientes['OC'] != oc]

            asunto = f"Factura {factura}"
            cuerpo = f"Estimados, saludos adjunto factura nro {factura} correspondiente a la OC {oc}" #  *CUERPO DE CORREO* 
            destinatario = 'aaguanangap@corporaciongpf.com'
            ruta_xml = ruta_archivo
            ruta_pdf = ruta_archivo.replace('.xml', '.pdf')
            enviar_correo(asunto, cuerpo, destinatario, [ruta_xml, ruta_pdf])

    if not dataframe_total.empty:
        if os.path.exists(ruta_excel_salida):
            book = openpyxl.load_workbook(ruta_excel_salida)
            with pd.ExcelWriter(ruta_excel_salida, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                writer.book = book
                startrow = writer.sheets['Sheet1'].max_row
                dataframe_total.to_excel(writer, sheet_name='Sheet1', index=False, header=False, startrow=startrow)
        else:
            with pd.ExcelWriter(ruta_excel_salida, engine='openpyxl', mode='w') as writer:
                dataframe_total.to_excel(writer, sheet_name='Sheet1', index=False)
    else:
        print("No hay nuevas informaciones para agregar.")
        
    with open(pickle_file, 'wb') as f:
        pickle.dump(facturas_procesadas, f)
    df_oc_pendientes.to_csv(csv_oc_pendientes, index=False)

def crear_excel_con_tabla(ruta_entrada, ruta_salida):
    """
    Carga un archivo Excel existente y crea un nuevo archivo Excel con los datos presentes en el primero,
    pero formateando los datos en forma de tabla de Excel. Este enfoque permite mejorar la visualización
    y el manejo de los datos en Excel, proporcionando funcionalidades de tabla como filtros y ordenamiento.

    Parámetros:
        ruta_entrada (str): La ruta al archivo Excel existente del cual se cargarán los datos.
        ruta_salida (str): La ruta donde se guardará el nuevo archivo Excel con los datos formateados como tabla.

    Funcionamiento:
        - Utiliza pandas para cargar el DataFrame desde el archivo Excel especificado en 'ruta_entrada'.
        - Crea un nuevo archivo Excel en la ubicación especificada por 'ruta_salida', utilizando XlsxWriter
            como motor para permitir la personalización y adición de la tabla.
        - Agrega el DataFrame cargado al nuevo archivo Excel, configurando los datos como una tabla Excel
            para facilitar su análisis y manipulación en Excel.
        - Configura los encabezados de columna de la tabla basándose en los nombres de las columnas del DataFrame.

    Notas:
        - La tabla creada en el archivo Excel tendrá funcionalidades como el filtrado automático y la capacidad
            de ordenar las columnas, mejorando significativamente la interacción con los datos.
        - Se asume que el archivo de entrada es un archivo Excel válido y que pandas puede leerlo sin problemas.
        - La creación de la tabla se realiza con la biblioteca XlsxWriter, lo que implica que este paquete debe
            estar instalado en el entorno de ejecución.
    """
    df = pd.read_excel(ruta_entrada)  # Carga el DataFrame desde el archivo de entrada.

    with pd.ExcelWriter(ruta_salida, engine='xlsxwriter') as writer:  # Crea un nuevo archivo Excel.
        df.to_excel(writer, sheet_name='Sheet1', index=False, header=True)  # Añade el DataFrame al archivo Excel.

        workbook = writer.book  # Obtiene el libro de trabajo de XlsxWriter.
        worksheet = writer.sheets['Sheet1']  # Obtiene la hoja de trabajo.

        # Configura y añade la tabla en el Excel, usando los nombres de columnas del DataFrame como encabezados.
        worksheet.add_table(0, 0, df.shape[0], df.shape[1] - 1,
                            {'columns': [{'header': col} for col in df.columns]})

def cargar_y_mapear_terceros(ruta_terceros_csv):
    """
    Carga un DataFrame de un archivo CSV, normaliza los RUC y crea un diccionario mapeando RUC a TERCERO.

    Parámetros:
    - ruta_terceros_csv: La ruta al archivo CSV que contiene la información de terceros.

    Retorna:
    - mapeo_terceros: Un diccionario donde las claves son los RUC normalizados y los valores son los TERCERO correspondientes.
    """
    # Intenta leer el archivo CSV con diferentes codecs
    try:
        terceros_df = pd.read_csv(ruta_terceros_csv, encoding='utf-8')
    except UnicodeDecodeError:
        terceros_df = pd.read_csv(ruta_terceros_csv, encoding='latin1')  # Prueba con el codec latin1

    terceros_df['RUC'] = terceros_df['RUC'].apply(lambda x: normalizar_ruc(str(x)))
    #mapeo_terceros = terceros_df.set_index('RUC')['TERCERO'].to_dict()
    terceros_df.drop_duplicates(subset='RUC', inplace=True)
    mapeo_terceros = terceros_df.set_index('RUC')[['TERCERO', 'CC', 'NOMBRE FARMACIA', 'FACTURA SEMESTRAL/MENSUAL']].to_dict(orient='index')


    return mapeo_terceros


def main():
    """
    Función principal que coordina la ejecución de tareas relacionadas con el procesamiento de facturas,
    incluyendo el mapeo de terceros, el registro y limpieza de carpetas vacías, la actualización de una tabla
    Excel con información de facturas y la creación de un archivo Excel formateado como tabla.

    Funcionamiento:
        - Inicialmente, ejecuta la función 'cargar_y_mapear_terceros' para preparar el mapeo de terceros
            a partir de un archivo CSV.
        - Programa tareas recurrentes para registrar carpetas vacías, limpiar registros de carpetas, actualizar
            la tabla Excel con información extraída de archivos XML, y finalmente, crear un archivo Excel con los
            datos formateados como tabla.
        - Estas tareas se ejecutan en un bucle infinito, con cada tarea programada para ejecutarse cada 10 segundos.
        - La ejecución del bucle se puede interrumpir manualmente por el usuario mediante una interrupción de teclado (Ctrl+C),
            lo cual detiene el programa de manera segura.

    Notas:
        - La función depende de las variables globales 'ruta_terceros_csv' y 'ruta_excel_salida', las cuales deben estar
            correctamente definidas y apuntar a las rutas de los archivos relevantes.
        - Utiliza la biblioteca 'schedule' para programar la ejecución periódica de las tareas. Esta biblioteca debe
            estar instalada en el entorno de ejecución.
        - La interacción del usuario mediante una interrupción de teclado es capturada para terminar el programa,
            lo que proporciona una forma segura de detener la ejecución.
    """
    # Ejecuta la función para cargar_y_mapear_terceros
    cargar_y_mapear_terceros(ruta_terceros_csv)

    # Programa las otras tareas para ejecución periódica
    schedule.every(10).seconds.do(registrar_carpetas_vacias)
    schedule.every(10).seconds.do(limpiar_registros_carpetas)
    schedule.every(10).seconds.do(actualizar_tabla_excel_y_limpieza, ruta_excel_salida)
    schedule.every(10).seconds.do(crear_excel_con_tabla, ruta_excel_salida, os.path.join(current_dir, 'OCS\\salida_tabla.xlsx'))

    # Bucle infinito para mantener en ejecución las tareas programadas
    try:
        while True:
            schedule.run_pending()  # Ejecuta las tareas pendientes según su programación.
            time.sleep(1)  # Espera 1 segundo antes de la próxima verificación de tareas pendientes.
    except KeyboardInterrupt:
        print("Programa terminado por el usuario.")  # Mensaje de salida al detener el programa manualmente.



# Ejecución
main()

No hay nuevas informaciones para agregar.
No hay nuevas informaciones para agregar.
No hay nuevas informaciones para agregar.
Programa terminado por el usuario.
